# 1. 크롤링과 함께 쓰는 mongoDB 예제

### cine21 인물 랭킹 알아내기
  - http://www.cine21.com/rank/person/
  - 조회를 눌렀을 때, Go to Network -> content 의 Request URL/Method 와 Form 데이터 알아내기
    - Request URL: http://www.cine21.com/rank/person/content
    - Request Method: POST
    - Form Data
      - section:actor
      - period_start:2021-10
      - gender:all
      - page:1
    - 하단부 페이지를 누를 때마다, Form Data 의 page 값이 바뀜

In [1]:
import requests as req
import re
import datetime
from bs4 import BeautifulSoup as bs
from pymongo import MongoClient
import pymongo

In [2]:
# 몽고DB 서버 연결
username = 'hans'
password = '1234'
conn = pymongo.MongoClient('mongodb://%s:%s@127.0.0.1' % (username, password))

In [3]:
# db, collection 연결(없으면 생성)
# python과 연동되었을 때, db와 collection의 경우, document insert 싯점에 생성됨
mongodb = conn.cine21
actor_collection = mongodb.actor_collection

In [4]:
# actor 정보가 있는지 확인하기
actor_list = actor_collection.find()
for actor in actor_list:
    print(actor['actor'])

In [5]:
# request url http://www.cine21.com/rank/person
def get_pages(page):
    cine21_url = 'http://www.cine21.com/rank/person/content'
    month = "2021-10"
    # post 방식으로 데이터를 가져오기 위한 request 시 필요 정보 F12/network 의 content/header에서 url과 form data 확인함.
    conditions = dict()
    conditions['section'] = 'actor'
    conditions['period_start'] = month
    conditions['gender'] = 'all'
    conditions['page'] = page 

    # post 방식으로 request 요청
    res = req.post(cine21_url, data = conditions)
    print(conditions)
    soup = bs(res.text, "html.parser")
    return soup

In [6]:
def get_items(soup):
    actor=[]
    urls=[]
    tds = soup.select("div.name>a")
    for td in tds:
        actor.append(td.get_text(strip=True).split("(")[0])
        urls.append(td["href"])

    score = []
    tds = soup.select("li>strong")
    for td in tds:
        score.append(td.get_text(strip=True).replace(",",""))

    movie =[]
    tds = soup.select("li.people_li>ul.mov_list>li>a>span")  
    for td in tds:
        movie.append(td.get_text(strip=True))

    print(actor, score, movie, urls)   
    return actor, score, movie, urls 

In [7]:
 
def put_data(actor, score, movie, urls):
    for i, j, k, l in zip(actor, score, movie, urls):
        post ={}
        post['actor'] = i
        post['score'] = j
        post['movie'] = k

        cine21_url="http://www.cine21.com" + l
        #print(cine21_url)
        res = req.post(cine21_url)
        soup = bs(res.text, "html.parser")
        tds = soup.select("ul.default_info>li")
        dic={}
        for td in tds:
    #        print(td)
            key = (td.select("li>span")[0].get_text(strip = True))
            value = (td.get_text(strip = True)).split(key)[1]
            dic[key] = value   
        post['basic_info'] = dic
        actor_collection.insert_one(post)    

In [8]:
# page = int(input("몇 page data를 가져올까요? :"))
page = 1
for i in range(1,page + 1):
    print(page)
    soup = get_pages(page)
    actor, score, movie, urls = get_items(soup)
    put_data(actor, score, movie, urls)



1
{'section': 'actor', 'period_start': '2021-10', 'gender': 'all', 'page': 1}
['박정민', '이성민', '임윤아', '노회찬', '이수경', '박영남', '김환진'] ['161725', '121294', '80862', '51594', '40431', '27908', '24419'] ['기적', '기적', '기적', '노회찬6411', '기적', '극장판 짱구는 못말려: 격돌! 낙서왕국과 얼추 네 명의 용사들', '극장판 짱구는 못말려: 격돌! 낙서왕국과 얼추 네 명의 용사들'] ['/db/person/info/?person_id=74826', '/db/person/info/?person_id=25123', '/db/person/info/?person_id=66313', '/db/person/info/?person_id=70020', '/db/person/info/?person_id=96165', '/db/person/info/?person_id=32987', '/db/person/info/?person_id=74727']


NameError: name 'conditions' is not defined

In [ ]:
# 정규 표현식 참고 : https://regexr.com/
# 문자, 숫자를 찾음 : \w
# 특수 기호 () 문자로 인식 : \( \)

# 2. 반복 표현 ?, *, +
# + : 앞 문자가 1번 또는 그 이상 반복되는 패턴
# ? : 앞 문자가 0번 또는 1번 표시되는 패턴(없어도 되고, 한번 있어도 되는 패턴)
# * : 앞 문자가 0번 또는 그이상 반복되는 패턴
import re
test_data = '마동석(17편)'
#re.sub('정규표현식', '변경데이터', test_data)
print(re.sub("\(\w+\)", "", test_data))

마동석


In [ ]:
# [실습]
# 한 페이지 내의 배우이름만 가져오기
import re



### 각 배우별 상세 정보를 document에 넣고 싶다.
* 각 배우별 상세 정보를 별도 컬럼으로 만들려했더니, 각 배우별 상세 정보 항목이 다르다!
* 모든 상세 정보 항목을 컬럼으로 만들고, 각 컬럼에 매칭되는 컬럼값을 넣기가 쉽지 않다. 코드도 복잡하고!
* Mongodb는 NoSQL -> 통째로 집어넣자.!

* embedded document
  - document 의 컬럼값으로 document를 넣을 수 있다.

### 흥행지수 뽑기

### 각 배우별 출연 영화를 document에 저장하고 싶다.
  - 출연 영화는 한 개가 될 수도 있고, 여러 개가 될 수도 있음
  - 파이썬은 리스트, mongodb document는 컬럼에 배열(array)로 넣으면 됨 

### insert_one() 로 하나씩 데이터 입력하기 (반복문과 함께 사용하면, 여러 데이터를 넣을 수 있음)

- actor_list: 배우 이름
- actor_details: 배우 상세 정보
- actor_rate: 흥행 지수
- date: 기준월
- movie_list: 출연 영화 리스트!

### collection 삭제하기

In [ ]:
# collection drop
actor_collection.drop()

In [ ]:
# collection 찾기
docs = actor_collection.find()
docs

In [ ]:
for doc in docs:
    print(doc)

### Dictionary 타입으로 만들어서 한번에 insert_many() 로 데이터 입력하기

### Update (컬럼명 변경 예제)

### 컬렉션 객체 이름도 바꿀 수 있겠지요
actor_collection -> actors_info 로 변경